# preprocess_annon_file

for pan_omics project

04/09/2020

In [148]:
import pandas as pd
import subprocess
import pybedtools

In [159]:
tss_file = 'TSS_annon_ALL.csv'
gencode_file = 'gene_annon_ALL.csv'


In [160]:
# make gencode list
gencode_df = pd.read_csv(gencode_file,index_col=0)
gencode_df.columns = ['chr', 'start', 'end', 'width', 'strand', 'gene']
gencode_df['chr'] = 'chr' + gencode_df['chr'].astype(str)
gencode_df[:5]

,chr,start,end,width,strand,gene
1,chr1,69091,70008,918,*,OR4F5
2,chr1,134901,139379,4479,*,AL627309.1
3,chr1,367640,368634,995,*,OR4F29
4,chr1,621059,622053,995,*,OR4F16
5,chr1,738532,739137,606,*,AL669831.1


In [152]:
# make tss list
tss_df = pd.read_csv(tss_file,index_col=0)
tss_df.columns = ['chr', 'start', 'end', 'width', 'strand', 'TSS']
tss_df['chr'] = 'chr' + tss_df['chr'].astype(str)
tss_df.sort_values('chr',inplace=True)
tss_df.reset_index(drop=True,inplace=True)
tss_df[:5]

,chr,start,end,width,strand,TSS
0,chr1,1846266,1846266,1,*,CALML6
1,chr1,231376933,231376933,1,*,C1orf131
2,chr1,241683061,241683061,1,*,FH
3,chr1,120202421,120202421,1,*,PHGDH
4,chr1,204135465,204135465,1,*,REN


In [153]:
check_tss_unique = pd.DataFrame(tss_df.groupby("TSS")['end'].nunique())
print(check_tss_unique.shape)
print(check_tss_unique[check_tss_unique.end>1].index.shape)
check_tss_unique[check_tss_unique.end>1].index.sort_values()

(20178, 1)
(69,)


Index(['ACE', 'ALG9', 'ANKRD18A', 'CCDC177', 'CDRT1', 'CELF6', 'CHTF8',
       'CKS1B', 'CLN3', 'CRIP1', 'CRYBG3', 'DEFB130', 'FAM47E-STBD1', 'GALT',
       'GOLGA7B', 'HOXC4', 'IER3IP1', 'ITFG3', 'ITGB3', 'KBTBD4', 'KIAA0391',
       'KLK9', 'KRBOX1', 'KRT222', 'LCN6', 'LIMS3', 'LSP1', 'LTB4R2', 'MDGA2',
       'MFRP', 'MRPL30', 'MRPS17', 'MUC3A', 'NAA60', 'NDUFA7', 'NDUFB8',
       'NPIPA7', 'OC90', 'PAGR1', 'PIK3R2', 'POLR2J2', 'PSMA1', 'PSMA2',
       'RBL1', 'RNASE11', 'RPP14', 'SDHD', 'SEPT1', 'SHPK', 'SIGLEC5',
       'SLC25A10', 'SNURF', 'SOGA3', 'TAP2', 'TIMM10B', 'TLR9', 'TM4SF19',
       'TMBIM4', 'TMEM236', 'TUBB3', 'UGT2A1', 'UQCR11', 'ZFP41', 'ZNF223',
       'ZNF286A', 'ZNF668', 'ZNF709', 'ZNF747', 'ZNF8'],
      dtype='object', name='TSS')

 note that there are 69 of 20178 unique TSS symbols that have two TSS locations, there is some discrepancy as to where those TSS's actually are located so we'll just use both and not this for future reference

In [209]:
PROMOTER_LEN_UPSTREAM=2000
PROMOTER_LEN_DOWNSTREAM=500

In [210]:
promoter_df = tss_df
promoter_df['start_n']= promoter_df['start'].map(int) - PROMOTER_LEN_UPSTREAM
promoter_df['end_n']= promoter_df.end + PROMOTER_LEN_DOWNSTREAM
promoter_df = promoter_df[['chr','start_n','end_n','TSS']]

In [211]:
promoter_df[:5]

,chr,start_n,end_n,TSS
0,chr1,1834266,1836766,CALML6
1,chr1,231364933,231367433,C1orf131
2,chr1,241671061,241673561,FH
3,chr1,120190421,120192921,PHGDH
4,chr1,204123465,204125965,REN


In [212]:
promoter_df.to_csv('promoter_hg19_{}_{}.bed'.format(PROMOTER_LEN_UPSTREAM,PROMOTER_LEN_DOWNSTREAM),
                   header=None,index=None,sep='\t')
                   

In [213]:
# cmd = "bedtools sort -i promoter_hg19_5000_5000.bed"
# cmd_arr = cmd.split(' ')
# print(cmd_arr)
# with open("promoter_hg19_2500_500_sort.bed", "w") as file:
#     subprocess.run(cmd_arr, shell=True, check=True, stdout=file)

In [214]:
promoter = pybedtools.BedTool('promoter_hg19_{}_{}.bed'.format(PROMOTER_LEN_UPSTREAM,PROMOTER_LEN_DOWNSTREAM))

promoter.sort().saveas('promoter_hg19_{}_{}_sort.bed'.format(PROMOTER_LEN_UPSTREAM,PROMOTER_LEN_DOWNSTREAM))



<BedTool(promoter_hg19_2000_500_sort.bed)>

# fix tf annotation file

In [47]:
rna_tpm_file = '../rnaseq/tissue_tpm_sym.csv'
rna_df = pd.read_csv(rna_tpm_file, index_col=0,header=0)
rna_df.index = [x.upper() for x in rna_df.index.values]
def get_exp(rna_df, sym, tissue,verbose=True):
    try:
        return rna_df.loc[sym,tissue]
    except KeyError:
        if verbose:
            print(sym, 'not found')
        return 0


In [72]:
hoco_annon_file = 'HOCOMOCOv11_core_annotation_HUMAN_mono.tsv'
hoco_annon = pd.read_table(hoco_annon_file,header=0)
# hoco_annon[['motif_abbr','motif_info']]=hoco_annon['Model'].str.split("_",expand=True)
hoco_id_to_tf = pd.Series(hoco_annon['Transcription factor'].values, index=hoco_annon.Model.values).to_dict()
# hoco_df = pd.read_table(hoco_annon_file, index_col=None, 
#                             names=['anchor_chr', 'anchor_start', 'anchor_end', 
#                                     'foot_chr', 'foot_start', 'foot_end', 'motif', 'score', 'strand'])
# hoco_df['transcription_factor'] = hoco_df['motif'].map(hoco_motif_to_tf)

In [73]:
# check to see if the rna expression for these tfs is in the table
for motif, tf in hoco_id_to_tf.items():
    exp = get_exp(rna_df, tf, 'GDSD0')
    if type(exp)==int:
        print(motif, tf)
        

T not found
BRAC_HUMAN.H11MO.0.A T
HNF1B not found
HNF1B_HUMAN.H11MO.0.A HNF1B
MEF2B not found
MEF2B_HUMAN.H11MO.0.A MEF2B
NR2E3 not found
NR2E3_HUMAN.H11MO.0.C NR2E3
ZNF8 not found
ZNF8_HUMAN.H11MO.0.C ZNF8


In [74]:
len(hoco_motif_to_tf)
401

401

So of the 401 human Transcription factors in HOCOMOCO,5 are not found in RNA_df by direct lookup, By manual search we find the following mappings

-  BRAC_HUMAN.H11MO.0.A, T --> TBXT
- HNF1B_HUMAN.H11MO.0.A --> n/a
- MEF2B_HUMAN.H11MO.0.A --> n/a
- NR2E3_HUMAN.H11MO.0.C --> n/a
- ZNF8_HUMAN.H11MO.0.C --> ZNF8-ERVK3-1

In [75]:
# manual correction of mapping dict
hoco_id_to_tf['BRAC_HUMAN.H11MO.0.A'] = 'TBXT'
hoco_id_to_tf['ZNF8_HUMAN.H11MO.0.C'] = 'ZNF8-ERVK3-1'
for motif, tf in hoco_id_to_tf.items():
    exp = get_exp(rna_df, tf, 'GDSD0')
    if type(exp)==int:
        print(motif, tf)
        

HNF1B not found
HNF1B_HUMAN.H11MO.0.A HNF1B
MEF2B not found
MEF2B_HUMAN.H11MO.0.A MEF2B
NR2E3 not found
NR2E3_HUMAN.H11MO.0.C NR2E3


In [76]:
hoco_id_to_tf = pd.DataFrame(pd.Series(hoco_id_to_tf).reset_index())
hoco_id_to_tf.columns=['id','tf']


In [78]:
hoco_id_to_tf[:5]

,id,tf
0,AHR_HUMAN.H11MO.0.B,AHR
1,AIRE_HUMAN.H11MO.0.C,AIRE
2,ALX1_HUMAN.H11MO.0.B,ALX1
3,ANDR_HUMAN.H11MO.0.A,AR
4,AP2A_HUMAN.H11MO.0.A,TFAP2A


In [79]:
hoco_id_to_tf.to_csv('HOCOMOCOv11_annotation.csv')